### Imports

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from IPython.display import Audio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa 

import os
import csv

### Predefined classifier function

In [4]:
def InstrumentClassifier(train_data, train_labels, test_data):
    clf_final = RandomForestClassifier(n_estimators=250)
    clf_final.fit(train_data, train_labels)
    labels = clf_final.predict(test_data)
    return labels

### Feature extraction (mfcc)

In [5]:
#https://dagshub.com/Ananya-github/Audio-signal-classification/src/main/Feature%20Extraction.ipynb
header = "filename "
for i in range(0, 13):
    header += f' mfcc_{i}'
header += ' label'
header = header.split()
file = open('Extracted_Features.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)

Instruments = 'gel pia voi cel'.split()
for i in Instruments:
    for filename in os.listdir(f'./data/{i}'):
        songname = f'./data/{i}/{filename}'
        y, sr = librosa.load(songname)
        to_append = f'{filename}'            
        mfcc = librosa.feature.mfcc(y=y,n_mfcc=13, sr=sr)
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {i}'
        file = open('Extracted_Features.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

### Splitting into training/test data/labels

In [34]:
df=pd.read_csv("Extracted_Features.csv")
traindata, testdata = train_test_split(df, test_size=0.01)
labels_traindata=traindata["label"]
traindata=traindata.drop(['filename',"label"],axis=1)
labels_testdata=testdata["label"]
testdata=testdata.drop(['filename',"label"],axis=1)
n_train = np.shape(traindata)[0]
n_test = np.shape(testdata)[0]
print(np.shape(traindata))
print(np.shape(testdata))

(2620, 13)
(27, 13)


### Training and error

In [36]:
classified_test_data = InstrumentClassifier(traindata, labels_traindata, testdata)
labels_test_np = labels_testdata.to_numpy()
not_equal = np.not_equal(labels_test_np, classified_test_data)
print("Incorrect: ", not_equal.sum(), " of ", n_test)
P = not_equal.sum()/(n_test)
print(P)

Incorrect:  4  of  27
0.14814814814814814
